## Project 3: Neural Networks and kNN
### Experimentation with H2o- First Dataset (Facebook Comments)

Research on h20:
<br> http://docs.h2o.ai/h2o-tutorials/latest-stable/tutorials/deeplearning/index.html
<br> https://github.com/h2oai/h2o-tutorials/blob/master/tutorials/deeplearning/deeplearning.py

#### Import packages.

In [1]:
import pandas as pd
import numpy as np
import random as rand
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import classification_report

In [3]:
import h2o
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

#### Import Data Set 1.

In [4]:
facebook = pd.read_csv('Features_Variant_1.csv', header=None)
test1 = pd.read_csv('Test_Case_1.csv', header=None)
test2 = pd.read_csv('Test_Case_2.csv', header=None)
test3 = pd.read_csv('Test_case_3.csv', header=None)
test4 = pd.read_csv('Test_case_4.csv', header=None)
test5 = pd.read_csv('Test_case_5.csv', header=None)
test6 = pd.read_csv('Test_case_6.csv', header=None)
test7 = pd.read_csv('Test_case_7.csv', header=None)
test8 = pd.read_csv('Test_case_8.csv', header=None)
test9 = pd.read_csv('Test_case_9.csv', header=None)
test10 = pd.read_csv('Test_case_10.csv', header=None)

In [5]:
facebook = facebook.append([test1, test2, test3, test4, test5, test6, test7, test8, test9, test10], ignore_index=True)

In [6]:
new_names = ['PagePopularity_1', 'PageCheckins_2', 'PageTalkingAbout_3', 'PageCategory_4']
old_names = facebook.columns[0:4]
facebook.rename(columns=dict(zip(old_names, new_names)), inplace=True)

new_names2 = ['CommentsBeforeBase_30', 'CommentsLast24h_31', 'CommentsLast48to24h_32', 'CommentsFirst24h_33', 'CommentsDiffFirstSecondDay_34', 'BaseTime_35',
'PostLength_36', 'PostShareCnt_37', 'PostPromoStatus_38', 'HoursReceived_39', 'SundayPublished_40', 'MondayPublished_41', 'TuesdayPublisher_42',
'WednesdayPublished_43', 'ThursdayPublished_44', 'FridayPublished_45', 'SaturdayPublished_46', 'SundayBase_47', 'MondayBase_48', 'TuesdayBase_49',
'WednesdayBase_50', 'ThursdayBase_51', 'FridayBase_52', 'SaturdayBase_53','CommentsNextH_Hours_54']
old_names2 = facebook.columns[29:54]
facebook.rename(columns=dict(zip(old_names2, new_names2)), inplace=True)

facebook2 = facebook.iloc[:, np.r_[0:4, 29:54]]
facebook2 = facebook2.drop('PostPromoStatus_38', axis=1)

#### Scale features for use in kNN.

In [7]:
mean_1 = sum(facebook2['PagePopularity_1'])/len(facebook2['PagePopularity_1'])
sd_1 = np.std(facebook2['PagePopularity_1'])
mean_2 = sum(facebook2['PageCheckins_2'])/len(facebook2['PageCheckins_2'])
sd_2 = np.std(facebook2['PageCheckins_2'])
mean_3 = sum(facebook2['PageTalkingAbout_3'])/len(facebook2['PageTalkingAbout_3'])
sd_3 = np.std(facebook2['PageTalkingAbout_3'])
mean_4 = sum(facebook2['PageCategory_4'])/len(facebook2['PageCategory_4'])
sd_4 = np.std(facebook2['PageCategory_4'])
mean_30 = sum(facebook2['CommentsBeforeBase_30'])/len(facebook2['CommentsBeforeBase_30'])
sd_30 = np.std(facebook2['CommentsBeforeBase_30'])
mean_31 = sum(facebook2['CommentsLast24h_31'])/len(facebook2['CommentsLast24h_31'])
sd_31 = np.std(facebook2['CommentsLast24h_31'])
mean_32= sum(facebook2['CommentsLast48to24h_32'])/len(facebook2['CommentsLast48to24h_32'])
sd_32 = np.std(facebook2['CommentsLast48to24h_32'])
mean_33= sum(facebook2['CommentsFirst24h_33'])/len(facebook2['CommentsFirst24h_33'])
sd_33 = np.std(facebook2['CommentsFirst24h_33'])
mean_34 = sum(facebook2['CommentsDiffFirstSecondDay_34'])/len(facebook2['CommentsDiffFirstSecondDay_34'])
sd_34 = np.std(facebook2['CommentsDiffFirstSecondDay_34'])
### find out if you should scale BaseTime 
mean_36 = sum(facebook2['PostLength_36'])/len(facebook2['PostLength_36'])
sd_36 = np.std(facebook2['PostLength_36'])
mean_37 = sum(facebook2['PostShareCnt_37'])/len(facebook2['PostShareCnt_37'])
sd_37 = np.std(facebook2['PostShareCnt_37'])
### find out if you should scale HoursReceived 
mean_39 = sum(facebook2['HoursReceived_39'])/len(facebook2['HoursReceived_39'])
sd_39 = np.std(facebook2['HoursReceived_39'])
mean_40 = sum(facebook2['SundayPublished_40'])/len(facebook2['SundayPublished_40'])
sd_40 = np.std(facebook2['SundayPublished_40'])
mean_41 = sum(facebook2['MondayPublished_41'])/len(facebook2['MondayPublished_41'])
sd_41 = np.std(facebook2['MondayPublished_41'])
mean_42 = sum(facebook2['TuesdayPublisher_42'])/len(facebook2['TuesdayPublisher_42'])
sd_42 = np.std(facebook2['TuesdayPublisher_42'])
mean_43 = sum(facebook2['WednesdayPublished_43'])/len(facebook2['WednesdayPublished_43'])
sd_43 = np.std(facebook2['WednesdayPublished_43'])
mean_44 = sum(facebook2['ThursdayPublished_44'])/len(facebook2['ThursdayPublished_44'])
sd_44 = np.std(facebook2['ThursdayPublished_44'])
mean_45 = sum(facebook2['FridayPublished_45'])/len(facebook2['FridayPublished_45'])
sd_45 = np.std(facebook2['FridayPublished_45'])
mean_46 = sum(facebook2['SaturdayPublished_46'])/len(facebook2['SaturdayPublished_46'])
sd_46 = np.std(facebook2['SaturdayPublished_46'])
mean_47 = sum(facebook2['SundayBase_47'])/len(facebook2['SundayBase_47'])
sd_47 = np.std(facebook2['SundayBase_47'])
mean_48 = sum(facebook2['MondayBase_48'])/len(facebook2['MondayBase_48'])
sd_48 = np.std(facebook2['MondayBase_48'])
mean_49 = sum(facebook2['TuesdayBase_49'])/len(facebook2['TuesdayBase_49'])
sd_49 = np.std(facebook2['TuesdayBase_49'])
mean_50 = sum(facebook2['WednesdayBase_50'])/len(facebook2['WednesdayBase_50'])
sd_50 = np.std(facebook2['WednesdayBase_50'])
mean_51 = sum(facebook2['ThursdayBase_51'])/len(facebook2['ThursdayBase_51'])
sd_51 = np.std(facebook2['ThursdayBase_51'])
mean_52 = sum(facebook2['FridayBase_52'])/len(facebook2['FridayBase_52'])
sd_52 = np.std(facebook2['FridayBase_52'])
mean_53 = sum(facebook2['SaturdayBase_53'])/len(facebook2['SaturdayBase_53'])
sd_53 = np.std(facebook2['SaturdayBase_53'])
## DO NOT scale the target (54)

In [8]:
facebook2_s = facebook2.copy()

In [9]:
facebook2_s.head()

,PagePopularity_1,PageCheckins_2,PageTalkingAbout_3,PageCategory_4,CommentsBeforeBase_30,CommentsLast24h_31,CommentsLast48to24h_32,CommentsFirst24h_33,CommentsDiffFirstSecondDay_34,BaseTime_35,...,FridayPublished_45,SaturdayPublished_46,SundayBase_47,MondayBase_48,TuesdayBase_49,WednesdayBase_50,ThursdayBase_51,FridayBase_52,SaturdayBase_53,CommentsNextH_Hours_54
0,634995,0,463,1,0,0,0,0,0,65,...,0,0,0,0,0,0,0,0,1,0
1,634995,0,463,1,0,0,0,0,0,10,...,0,0,0,0,0,0,0,1,0,0
2,634995,0,463,1,0,0,0,0,0,14,...,1,0,0,0,0,0,0,0,1,0
3,634995,0,463,1,7,0,3,7,-3,62,...,1,0,0,1,0,0,0,0,0,0
4,634995,0,463,1,1,0,0,1,0,58,...,0,0,0,0,0,1,0,0,0,0


In [10]:
facebook2_s['PagePopularity_1'] = (facebook2['PagePopularity_1'] - mean_1)/sd_1
facebook2_s['PageCheckins_2'] = (facebook2['PageCheckins_2'] - mean_2)/sd_2
facebook2_s['PageTalkingAbout_3'] = (facebook2['PageTalkingAbout_3'] - mean_3)/sd_3
facebook2_s['PageCategory_4'] = (facebook2['PageCategory_4'] - mean_4)/sd_4
facebook2_s['CommentsBeforeBase_30'] = (facebook2['CommentsBeforeBase_30'] - mean_30)/sd_30
facebook2_s['CommentsLast24h_31'] = (facebook2['CommentsLast24h_31'] - mean_31)/sd_31
facebook2_s['CommentsLast48to24h_32'] = (facebook2['CommentsLast48to24h_32'] - mean_32)/sd_32
facebook2_s['CommentsFirst24h_33'] = (facebook2['CommentsFirst24h_33'] - mean_33)/sd_33
facebook2_s['CommentsDiffFirstSecondDay_34'] = (facebook2['CommentsDiffFirstSecondDay_34'] - mean_34)/sd_34
## check to see if 35 should be scaled
facebook2_s['BaseTime_35'] = facebook2['BaseTime_35']
facebook2_s['PostLength_36'] = (facebook2['PostLength_36'] - mean_36)/sd_36
facebook2_s['PostShareCnt_37'] = (facebook2['PostShareCnt_37'] - mean_37)/sd_37
##facebook2_s['PostPromoStatus_38'] = (facebook2['PostPromoStatus_38'] - mean_38)/sd_38 --- appears to be zero
## check to see if 39 should be scaled
facebook2_s['HoursReceived_39'] = (facebook2['HoursReceived_39'] - mean_39)/sd_39
facebook2_s['SundayPublished_40'] = (facebook2['SundayPublished_40'] - mean_40)/sd_40
facebook2_s['MondayPublished_41'] = (facebook2['MondayPublished_41'] - mean_41)/sd_41
facebook2_s['TuesdayPublisher_42'] = (facebook2['TuesdayPublisher_42'] - mean_42)/sd_42
facebook2_s['WednesdayPublished_43'] = (facebook2['WednesdayPublished_43'] - mean_43)/sd_43
facebook2_s['ThursdayPublished_44'] = (facebook2['ThursdayPublished_44'] - mean_44)/sd_44
facebook2_s['FridayPublished_45'] = (facebook2['FridayPublished_45'] - mean_45)/sd_45
facebook2_s['SaturdayPublished_46'] = (facebook2['SaturdayPublished_46'] - mean_46)/sd_46
facebook2_s['SundayBase_47'] = (facebook2['SundayBase_47'] - mean_47)/sd_47
facebook2_s['MondayBase_48'] = (facebook2['MondayBase_48'] - mean_48)/sd_48
facebook2_s['TuesdayBase_49'] = (facebook2['TuesdayBase_49'] - mean_49)/sd_49
facebook2_s['WednesdayBase_50'] = (facebook2['WednesdayBase_50'] - mean_50)/sd_50
facebook2_s['ThursdayBase_51'] = (facebook2['ThursdayBase_51'] - mean_51)/sd_51
facebook2_s['FridayBase_52'] = (facebook2['FridayBase_52'] - mean_52)/sd_52
facebook2_s['SaturdayBase_53'] = (facebook2['SaturdayBase_53'] - mean_53)/sd_53
## don't scales the target
facebook2_s['CommentsNextH_Hours_54'] = facebook2['CommentsNextH_Hours_54']

In [11]:
facebook2_s.head()

,PagePopularity_1,PageCheckins_2,PageTalkingAbout_3,PageCategory_4,CommentsBeforeBase_30,CommentsLast24h_31,CommentsLast48to24h_32,CommentsFirst24h_33,CommentsDiffFirstSecondDay_34,BaseTime_35,...,FridayPublished_45,SaturdayPublished_46,SundayBase_47,MondayBase_48,TuesdayBase_49,WednesdayBase_50,ThursdayBase_51,FridayBase_52,SaturdayBase_53,CommentsNextH_Hours_54
0,-0.107749,-0.220202,-0.359973,-1.161633,-0.396924,-0.277312,-0.269467,-0.400045,-0.036157,65,...,-0.417790,-0.40196,-0.413886,-0.393397,-0.393962,-0.411909,-0.418774,-0.411513,2.415662,0
1,-0.107749,-0.220202,-0.359973,-1.161633,-0.396924,-0.277312,-0.269467,-0.400045,-0.036157,10,...,-0.417790,-0.40196,-0.413886,-0.393397,-0.393962,-0.411909,-0.418774,2.430059,-0.413965,0
2,-0.107749,-0.220202,-0.359973,-1.161633,-0.396924,-0.277312,-0.269467,-0.400045,-0.036157,14,...,2.393547,-0.40196,-0.413886,-0.393397,-0.393962,-0.411909,-0.418774,-0.411513,2.415662,0
3,-0.107749,-0.220202,-0.359973,-1.161633,-0.348015,-0.277312,-0.228578,-0.347915,-0.066027,62,...,2.393547,-0.40196,-0.413886,2.541964,-0.393962,-0.411909,-0.418774,-0.411513,-0.413965,0
4,-0.107749,-0.220202,-0.359973,-1.161633,-0.389937,-0.277312,-0.269467,-0.392598,-0.036157,58,...,-0.417790,-0.40196,-0.413886,-0.393397,-0.393962,2.427723,-0.418774,-0.411513,-0.413965,0


#### Estlish binary classification label.

In [12]:
#### Create binary variable for classification analysis. Apply to both the scaled and unscaled data sets.

facebook2['Comment_Y'] = facebook2['CommentsNextH_Hours_54']>0
facebook2['Comment_Y'] = facebook2['Comment_Y'].apply(int)
facebook2_s['Comment_Y'] = facebook2['Comment_Y']

In [13]:
X = facebook2.iloc[:, 0:27]
y = facebook2.iloc[:, 28:29]
X_s = facebook2_s.iloc[:, 0:27]
y_s = pd.DataFrame(facebook2_s.iloc[:, 28:29])

In [14]:
facebook2_s.columns

Index(['PagePopularity_1', 'PageCheckins_2', 'PageTalkingAbout_3',
       'PageCategory_4', 'CommentsBeforeBase_30', 'CommentsLast24h_31',
       'CommentsLast48to24h_32', 'CommentsFirst24h_33',
       'CommentsDiffFirstSecondDay_34', 'BaseTime_35', 'PostLength_36',
       'PostShareCnt_37', 'HoursReceived_39', 'SundayPublished_40',
       'MondayPublished_41', 'TuesdayPublisher_42', 'WednesdayPublished_43',
       'ThursdayPublished_44', 'FridayPublished_45', 'SaturdayPublished_46',
       'SundayBase_47', 'MondayBase_48', 'TuesdayBase_49', 'WednesdayBase_50',
       'ThursdayBase_51', 'FridayBase_52', 'SaturdayBase_53',
       'CommentsNextH_Hours_54', 'Comment_Y'],
      dtype='object')

#### Split Data into testing and training.

In [15]:
from sklearn.model_selection import train_test_split

Xs_train, Xs_test, ys_train, ys_test = train_test_split(X_s, y_s, test_size=0.3, random_state=4786)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=4786)

In [16]:
h2o.init(ip="localhost", port=54321)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM (build 25.152-b12, mixed mode)56-b12)
  Starting server from C:\Users\katri\Anaconda3\h2o_jar\h2o.jar
  Ice root: C:\Users\katri\AppData\Local\Temp\tmpx1i6mbhp
  JVM stdout: C:\Users\katri\AppData\Local\Temp\tmpx1i6mbhp\h2o_katri_started_from_python.out
  JVM stderr: C:\Users\katri\AppData\Local\Temp\tmpx1i6mbhp\h2o_katri_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,03 secs
H2O cluster timezone:,America/Chicago
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.2
H2O cluster version age:,"1 year, 1 month and 6 days !!!"
H2O cluster name:,H2O_from_python_katri_ihmucp
H2O cluster total nodes:,1
H2O cluster free memory:,3.521 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [17]:
h2o.remove_all()    

### Experimentation with Deep Learning.

#### Baseline model. Using scaled data.

In [18]:
facebook2_s.head()

,PagePopularity_1,PageCheckins_2,PageTalkingAbout_3,PageCategory_4,CommentsBeforeBase_30,CommentsLast24h_31,CommentsLast48to24h_32,CommentsFirst24h_33,CommentsDiffFirstSecondDay_34,BaseTime_35,...,SaturdayPublished_46,SundayBase_47,MondayBase_48,TuesdayBase_49,WednesdayBase_50,ThursdayBase_51,FridayBase_52,SaturdayBase_53,CommentsNextH_Hours_54,Comment_Y
0,-0.107749,-0.220202,-0.359973,-1.161633,-0.396924,-0.277312,-0.269467,-0.400045,-0.036157,65,...,-0.40196,-0.413886,-0.393397,-0.393962,-0.411909,-0.418774,-0.411513,2.415662,0,0
1,-0.107749,-0.220202,-0.359973,-1.161633,-0.396924,-0.277312,-0.269467,-0.400045,-0.036157,10,...,-0.40196,-0.413886,-0.393397,-0.393962,-0.411909,-0.418774,2.430059,-0.413965,0,0
2,-0.107749,-0.220202,-0.359973,-1.161633,-0.396924,-0.277312,-0.269467,-0.400045,-0.036157,14,...,-0.40196,-0.413886,-0.393397,-0.393962,-0.411909,-0.418774,-0.411513,2.415662,0,0
3,-0.107749,-0.220202,-0.359973,-1.161633,-0.348015,-0.277312,-0.228578,-0.347915,-0.066027,62,...,-0.40196,-0.413886,2.541964,-0.393962,-0.411909,-0.418774,-0.411513,-0.413965,0,0
4,-0.107749,-0.220202,-0.359973,-1.161633,-0.389937,-0.277312,-0.269467,-0.392598,-0.036157,58,...,-0.40196,-0.413886,-0.393397,-0.393962,2.427723,-0.418774,-0.411513,-0.413965,0,0


In [19]:
facebook2_sHF = h2o.H2OFrame(facebook2_s)
facebook2_sHF['Comment_Y'] = facebook2_sHF['Comment_Y'].asfactor()

C:\Users\katri\Anaconda3\lib\site-packages\h2o\utils\shared_utils.py:170: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  data = _handle_python_lists(python_obj.as_matrix().tolist(), -1)[1]


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [20]:
train, valid, test = facebook2_sHF.split_frame([0.6, 0.2])

In [21]:
x = ['PagePopularity_1', 'PageCheckins_2', 'PageTalkingAbout_3',
       'PageCategory_4', 'CommentsBeforeBase_30', 'CommentsLast24h_31',
       'CommentsLast48to24h_32', 'CommentsFirst24h_33',
       'CommentsDiffFirstSecondDay_34', 'BaseTime_35', 'PostLength_36',
       'PostShareCnt_37', 'HoursReceived_39', 'SundayPublished_40',
       'MondayPublished_41', 'TuesdayPublisher_42', 'WednesdayPublished_43',
       'ThursdayPublished_44', 'FridayPublished_45', 'SaturdayPublished_46',
       'SundayBase_47', 'MondayBase_48', 'TuesdayBase_49', 'WednesdayBase_50',
       'ThursdayBase_51', 'FridayBase_52', 'SaturdayBase_53']
y = 'Comment_Y'

In [22]:
DLbaseModel = h2o.estimators.H2ODeepLearningEstimator(model_id="DL_defaults")
DLbaseModel.train(x, y, train)

deeplearning Model Build progress: |██████████████████████████████████████| 100%


In [23]:
print(DLbaseModel)

Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  DL_defaults


ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.12515961430994368
RMSE: 0.35377904730204657
LogLoss: 0.3924370587405597
Mean Per-Class Error: 0.17946852297574845
AUC: 0.9022612390065192
Gini: 0.8045224780130384
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3991753454073841: 


,0,1,Error,Rate
0,4546.0,833.0,0.1549,(833.0/5379.0)
1,922.0,3584.0,0.2046,(922.0/4506.0)
Total,5468.0,4417.0,0.1775,(1755.0/9885.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.3991753,0.8033173,212.0
max f2,0.0943526,0.8591532,338.0
max f0point5,0.6563819,0.8423822,130.0
max accuracy,0.4681476,0.8256955,187.0
max precision,0.9999098,1.0,0.0
max recall,0.0169609,1.0,394.0
max specificity,0.9999098,1.0,0.0
max absolute_mcc,0.5226548,0.6497899,170.0
max min_per_class_accuracy,0.3528172,0.8163227,230.0
max mean_per_class_accuracy,0.4619272,0.8205315,189.0


Gains/Lift Table: Avg response rate: 45.58 %



...


Scoring History: 


,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_auc,training_lift,training_classification_error
,2019-04-14 16:14:07,0.000 sec,None,0.0,0,0.0,nan,nan,nan,nan,nan
,2019-04-14 16:14:12,7.440 sec,7124 obs/sec,1.0,1,25051.0,0.3785506,0.4594076,0.8794146,2.1937417,0.1964593
,2019-04-14 16:14:28,21.634 sec,14491 obs/sec,10.0,10,250510.0,0.3537790,0.3924371,0.9022612,2.1937417,0.1775417


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
CommentsLast24h_31,1.0,1.0,0.0868025
PostShareCnt_37,0.5170279,0.5170279,0.0448793
ThursdayPublished_44,0.5086525,0.5086525,0.0441523
WednesdayPublished_43,0.5065908,0.5065908,0.0439734
CommentsLast48to24h_32,0.4965707,0.4965707,0.0431036
---,---,---,---
PagePopularity_1,0.2692086,0.2692086,0.0233680
PageCheckins_2,0.2379476,0.2379476,0.0206545
PageCategory_4,0.1868017,0.1868017,0.0162149
HoursReceived_39,0.1678796,0.1678796,0.0145724



See the whole table with table.as_data_frame()



In [24]:
DLbaseModel.scoring_history()

,,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_auc,training_lift,training_classification_error
0,,2019-04-14 16:14:07,0.000 sec,None,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN
1,,2019-04-14 16:14:12,7.440 sec,7124 obs/sec,1.0,1,25051.0,0.378551,0.459408,0.879415,2.193742,0.196459
2,,2019-04-14 16:14:28,21.634 sec,14491 obs/sec,10.0,10,250510.0,0.353779,0.392437,0.902261,2.193742,0.177542


In [25]:
# hyperparameters
dl_params1 = {'epochs': [5,10,15,20,25],
                'activation': ["Rectifier","Tanh","Maxout"],
             }

In [26]:
# Search criteria
search_criteria = {'strategy': 'RandomDiscrete', 'max_models': 50, 'seed': 3003}

# Train and validate a random grid of DL
dl_grid1 = H2OGridSearch(model=H2ODeepLearningEstimator(hidden=[50,50]),
                          grid_id='DLgrid1',
                          hyper_params=dl_params1,
                          search_criteria=search_criteria)

Note: limited hidden layers to 50,50 instead of default 200,200 to speed up learning

In [27]:
dl_grid1.train(x=x, y=y,
                training_frame=train,
                validation_frame=valid,
                seed=3003,
                variable_importances=True, 
                stopping_metric="misclassification")

deeplearning Grid Build progress: |███████████████████████████████████████| 100%


In [28]:
# Get the grid results, sorted by validation AUC
dl_gridperf = dl_grid1.get_grid(sort_by='auc', decreasing=True)

In [29]:
dl_gridperf

     activation epochs         model_ids                 auc
0          Tanh   20.0  DLgrid1_model_12  0.8989070062974911
1          Tanh   15.0   DLgrid1_model_0  0.8980348861819262
2          Tanh   25.0   DLgrid1_model_2  0.8975626432108803
3     Rectifier   20.0  DLgrid1_model_10  0.8963454224626279
4        Maxout   20.0   DLgrid1_model_7  0.8961097026906544
5        Maxout   15.0   DLgrid1_model_8  0.8953877086672338
6     Rectifier   10.0   DLgrid1_model_9  0.8950676003560789
7        Maxout   25.0  DLgrid1_model_14  0.8939501480371816
8        Maxout   10.0  DLgrid1_model_13  0.8937686022067158
9     Rectifier   15.0   DLgrid1_model_6  0.8927578909166369
10         Tanh   10.0   DLgrid1_model_1  0.8926681079383648
11         Tanh    5.0   DLgrid1_model_4   0.892511210103532
12    Rectifier   25.0  DLgrid1_model_11  0.8921824936448917
13    Rectifier    5.0   DLgrid1_model_3  0.8919208059613373
14       Maxout    5.0   DLgrid1_model_5  0.8870597850763646


In [30]:
# Grab the top GBM model, chosen by validation AUC
best_dl = dl_gridperf.models[0]

In [31]:
best_dl_perf = best_dl.model_performance(test)
print(best_dl_perf.auc())

0.9041581457620473


Check to see which model is most accuarate vs. most specific.

In [32]:
# Get the grid results, sorted by validation AUC
dl_gridperf_accuracy = dl_grid1.get_grid(sort_by='accuracy', decreasing=True)
dl_gridperf_precision = dl_grid1.get_grid(sort_by='precision', decreasing=True)

In [33]:
dl_gridperf_accuracy

     activation epochs         model_ids            accuracy
0     Rectifier   25.0  DLgrid1_model_11  0.8224767358625626
1        Maxout   20.0   DLgrid1_model_7  0.8219995227869243
2     Rectifier   20.0  DLgrid1_model_10     0.8204485802911
3          Tanh   25.0   DLgrid1_model_2  0.8197327606776426
4        Maxout   25.0  DLgrid1_model_14  0.8174659985683608
5     Rectifier   10.0   DLgrid1_model_9  0.8144834168456215
6          Tanh   20.0  DLgrid1_model_12  0.8141255070388929
7        Maxout    5.0   DLgrid1_model_5  0.8131710808876164
8        Maxout   15.0   DLgrid1_model_8  0.8128131710808876
9          Tanh   10.0   DLgrid1_model_1  0.8119780481985206
10         Tanh   15.0   DLgrid1_model_0  0.8100691958959676
11       Maxout   10.0  DLgrid1_model_13  0.8085182534001432
12    Rectifier    5.0   DLgrid1_model_3  0.8083989501312336
13    Rectifier   15.0   DLgrid1_model_6  0.8072059174421379
14         Tanh    5.0   DLgrid1_model_4  0.8000477213075639


In [34]:
dl_gridperf_precision

     activation epochs         model_ids           precision
0        Maxout   20.0   DLgrid1_model_7  0.8248603351955307
1     Rectifier   25.0  DLgrid1_model_11  0.8133405056481979
2          Tanh   25.0   DLgrid1_model_2  0.8045946659625033
3     Rectifier   20.0  DLgrid1_model_10  0.8045825651830393
4        Maxout   25.0  DLgrid1_model_14  0.8044172432144758
5        Maxout    5.0   DLgrid1_model_5  0.7909560723514212
6          Tanh   10.0   DLgrid1_model_1   0.780811623246493
7        Maxout   15.0   DLgrid1_model_8  0.7779975278121137
8     Rectifier   10.0   DLgrid1_model_9  0.7763125763125763
9          Tanh   20.0  DLgrid1_model_12  0.7733430091920658
10       Maxout   10.0  DLgrid1_model_13   0.773007648655317
11    Rectifier    5.0   DLgrid1_model_3  0.7680562287930198
12    Rectifier   15.0   DLgrid1_model_6  0.7633971291866029
13         Tanh   15.0   DLgrid1_model_0  0.7612676056338028
14         Tanh    5.0   DLgrid1_model_4  0.7403500897666068


In [35]:
tanh_model_h2o = H2ODeepLearningEstimator(
  model_id="sun_tanh",
  overwrite_with_best_model=False,
  hidden=[50,50], 
  epochs=20,
  activation='Tanh'
)
tanh_model_h2o.train(x, y, train)

deeplearning Model Build progress: |██████████████████████████████████████| 100%


In [36]:
tanh_model_h2o

Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  sun_tanh


ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.12164146672389753
RMSE: 0.3487713674083604
LogLoss: 0.38445633302660476
Mean Per-Class Error: 0.1761943752667674
AUC: 0.9060946794162161
Gini: 0.8121893588324323
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3973282879010616: 


,0,1,Error,Rate
0,4516.0,995.0,0.1805,(995.0/5511.0)
1,791.0,3791.0,0.1726,(791.0/4582.0)
Total,5307.0,4786.0,0.177,(1786.0/10093.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.3973283,0.8093510,219.0
max f2,0.1529697,0.8643440,316.0
max f0point5,0.7521450,0.8469434,105.0
max accuracy,0.5454268,0.8283959,171.0
max precision,0.9919664,0.9977629,3.0
max recall,0.0166360,1.0,398.0
max specificity,0.9936670,0.9998185,0.0
max absolute_mcc,0.5794797,0.6538968,161.0
max min_per_class_accuracy,0.4060587,0.8223553,216.0
max mean_per_class_accuracy,0.4383056,0.8238056,207.0


Gains/Lift Table: Avg response rate: 45.40 %



...


Scoring History: 


,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_auc,training_lift,training_classification_error
,2019-04-14 16:18:03,0.000 sec,None,0.0,0,0.0,nan,nan,nan,nan,nan
,2019-04-14 16:18:06,2.583 sec,20806 obs/sec,2.0,1,50102.0,0.3790352,0.4517104,0.8804103,2.2027499,0.2024175
,2019-04-14 16:18:11,7.870 sec,19764 obs/sec,6.0,3,150306.0,0.3623395,0.4111854,0.8946848,2.2027499,0.1820073
,2019-04-14 16:18:18,14.831 sec,20928 obs/sec,12.0,6,300612.0,0.3548816,0.3955657,0.8994674,2.2027499,0.1856732
,2019-04-14 16:18:25,21.511 sec,21519 obs/sec,18.0,9,450918.0,0.3543798,0.3955649,0.9013591,2.1809405,0.1804221
,2019-04-14 16:18:27,23.986 sec,21515 obs/sec,20.0,10,501020.0,0.3487714,0.3844563,0.9060947,2.1809405,0.1769543


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
CommentsLast24h_31,1.0,1.0,0.1098348
TuesdayPublisher_42,0.4411606,0.4411606,0.0484548
WednesdayPublished_43,0.4307599,0.4307599,0.0473124
MondayPublished_41,0.4224504,0.4224504,0.0463997
FridayBase_52,0.4189316,0.4189316,0.0460132
---,---,---,---
PagePopularity_1,0.1640440,0.1640440,0.0180177
PageCheckins_2,0.1397697,0.1397697,0.0153516
HoursReceived_39,0.1339969,0.1339969,0.0147175
PageCategory_4,0.1209436,0.1209436,0.0132838



See the whole table with table.as_data_frame()


In [37]:
print(tanh_model_h2o.auc())

0.9060946794162161


In [38]:
tanh_model_h2o.model_performance(test).auc()

0.9026712200598802

Check to see if accuracy improves with additional hidden layer.

In [39]:
tanh_model_h2o_2 = H2ODeepLearningEstimator(
  model_id="sun_tanh",
  overwrite_with_best_model=False,
  hidden=[25,25,25], 
  epochs=20,
  activation='Tanh'
)
tanh_model_h2o_2.train(x, y, train)

deeplearning Model Build progress: |██████████████████████████████████████| 100%


In [40]:
tanh_model_h2o_2.auc()

0.9019734566915967

In [41]:
tanh_model_h2o_2.model_performance(test).auc()

0.9015002361348731

Did not make much of a difference.

In [42]:
tanh_model_h2o_3 = H2ODeepLearningEstimator(
  model_id="dial_up_the_spf",
  overwrite_with_best_model=False,
  hidden=[50,50,50,50], 
  epochs=20,
  activation='Tanh'
)
tanh_model_h2o_3.train(x, y, train)

deeplearning Model Build progress: |██████████████████████████████████████| 100%


In [43]:
print(tanh_model_h2o_3.auc(), tanh_model_h2o_3.model_performance(test).auc())

0.8994841076109258 0.9007149762974052
